# 15、Dropout原理以及其TF&Torch&Numpy源码实现

- [Dropout 论文](https://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf)
- [RDropout 论文](https://arxiv.org/pdf/2106.14448)

In [1]:
import torch
import torch.nn as nn


In [7]:
# 使用 Drouout 类
m = nn.Dropout(p=0.4)
input = torch.randn(2, 3)
m(input) # 40%的节点置为0

tensor([[-0.1622, -0.0000, -1.2924],
        [ 1.6719, -0.2125, -0.9680]])

In [8]:
# 使用 Dropout 函数
dropout_func = nn.functional.dropout
dropout_func(input, p=0.5, training=True) # 如果不是training,那么dropout无效

tensor([[-0.0000, -0.0000, -1.5508],
        [ 0.0000, -0.2551, -1.1616]])

## 读论文

原来机器学习为了防止过拟合，都是使用集成学习，但是在运行的时候很耗费资源，于是hinton提出了了dropout来解决过拟合。

训练的时候，dropout会随机丢弃部分神经元，其实是在训练多个不同的网络。

![image-20250619165655657](http://assets.hypervoid.top/img/2025/06/19/image-20250619165655657-afa6.png)

应用dropout后，部分神经元会只和部分的神经元相连，


图二：测试时不失活，我们怎样在测试时逼近效果？
测试阶段，乘以一个权重p,保证输出的**期望**相同。（不过为了测试时越简单越好，那么可以改成在训练阶段乘以系数(1-p)）
![image-20250619165836319](http://assets.hypervoid.top/img/2025/06/19/image-20250619165836319-040e.png)

In [10]:
import numpy as np

# 这里的权重缩放在test阶段进行

def train(x, w1, b1, w2, b2, dropout_prob=0.5):
    ll1:np.ndarray = np.dot(w1, x) + b1
    ll1 = np.maximum(0, ll1)
    # 应用dropout
    mask = np.random.binomial(ll1.size, 1-dropout_prob)
    ll1 *= mask
    # 第二层
    ll2:np.ndarray = np.dot(w2, ll1) + b2
    ll2 = np.maximum(0, ll2)
    mask = np.random.binomial(ll2.size, 1-dropout_prob)
    ll2 *= mask
    return ll2

def test(x, w1, b1, w2, b2, dropout_prob):
    ll1:np.ndarray = np.dot(w1, x) + b1
    ll1 = np.maximum(0, ll1)
    # 应用dropout
    ll1 *= 1 - dropout_prob
    # 第二层
    ll2:np.ndarray = np.dot(w2, ll1) + b2
    ll2 = np.maximum(0, ll2)
    ll2 *= 1 - dropout_prob
    return ll2


In [ ]:
import numpy as np

# 这里的权重缩放在 train 阶段进行

def train(x, w1, b1, w2, b2, dropout_prob=0.5):
    ll1:np.ndarray = np.dot(w1, x) + b1
    ll1 = np.maximum(0, ll1)
    # 应用dropout
    mask = np.random.binomial(ll1.size, 1-dropout_prob)
    ll1 *= mask / (1-dropout_prob)
    # 第二层
    ll2:np.ndarray = np.dot(w2, ll1) + b2
    ll2 = np.maximum(0, ll2)
    mask = np.random.binomial(ll2.size, 1-dropout_prob)
    ll2 *= mask / (1-dropout_prob)
    return ll2

def test(x, w1, b1, w2, b2):
    ll1:np.ndarray = np.dot(w1, x) + b1
    ll1 = np.maximum(0, ll1)
    ll2:np.ndarray = np.dot(w2, ll1) + b2
    ll2 = np.maximum(0, ll2)
    return ll2


## R-Dropout

用于解决dropout中训练和测试不太一致的问题。此外对于不同的minibatch,实际上训练的是n个网络。
R-Dropout 在训练过程中，将同一个输入样本**两次**通过带有 Dropout 的模型，得到两个不同的输出概率分布，然后通过最小化这两个分布之间的**双向 KL 散度**，来约束这两个由 Dropout 随机生成的子模型，使其输出保持一致。

![image-20250619192916306](http://assets.hypervoid.top/img/2025/06/19/image-20250619192916306-61d7.png)

In [ ]:
import numpy as np

# R-Dropout 伪代码

def train(x, w1, b1, w2, b2, dropout_prob=0.5):
    x = torch.cat([x,x], 0)
    ll1:np.ndarray = np.dot(w1, x) + b1
    ll1 = np.maximum(0, ll1)
    # 应用dropout
    mask = np.random.binomial(ll1.size, 1-dropout_prob)
    ll1 *= mask / (1-dropout_prob)
    # 第二层
    ll2:np.ndarray = np.dot(w2, ll1) + b2
    ll2 = np.maximum(0, ll2)
    mask = np.random.binomial(ll2.size, 1-dropout_prob)
    ll2 *= mask / (1-dropout_prob)
    logits1, logits2 = some_func(ll2)
    nll_loss1 = nll(logits1, logits2)
    nll_loss2 = nll(logits2, logits1)
    kl_loss = kl(logits1, logits2)
    loss = nll_loss1 + nll_loss2 + kl_loss
    return loss

def test(x, w1, b1, w2, b2):
    ll1:np.ndarray = np.dot(w1, x) + b1
    ll1 = np.maximum(0, ll1)
    ll2:np.ndarray = np.dot(w2, ll1) + b2
    ll2 = np.maximum(0, ll2)
    return ll2
